# Addestrare RL per bilanciare il Cartpole

Questo notebook fa parte del [Curriculum AI per Principianti](http://aka.ms/ai-beginners). È stato ispirato dal [tutorial ufficiale di PyTorch](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) e da [questa implementazione di Cartpole in PyTorch](https://github.com/yc930401/Actor-Critic-pytorch).

In questo esempio, utilizzeremo RL per addestrare un modello a bilanciare un'asta su un carrello che può muoversi a sinistra e a destra su una scala orizzontale. Useremo l'ambiente [OpenAI Gym](https://www.gymlibrary.ml/) per simulare l'asta.

> **Nota**: Puoi eseguire il codice di questa lezione localmente (ad esempio, da Visual Studio Code), nel qual caso la simulazione si aprirà in una nuova finestra. Quando esegui il codice online, potrebbe essere necessario apportare alcune modifiche al codice, come descritto [qui](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Inizieremo assicurandoci che Gym sia installato:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Ora creiamo l'ambiente CartPole e vediamo come operare su di esso. Un ambiente ha le seguenti proprietà:

* **Action space** è l'insieme delle azioni possibili che possiamo eseguire a ogni passo della simulazione
* **Observation space** è lo spazio delle osservazioni che possiamo effettuare


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Vediamo come funziona la simulazione. Il seguente ciclo esegue la simulazione fino a quando `env.step` non restituisce il flag di terminazione `done`. Sceglieremo le azioni in modo casuale utilizzando `env.action_space.sample()`, il che significa che l'esperimento probabilmente fallirà molto rapidamente (l'ambiente CartPole termina quando la velocità del CartPole, la sua posizione o l'angolo superano certi limiti).

> La simulazione si aprirà in una nuova finestra. Puoi eseguire il codice più volte e osservare come si comporta.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Puoi notare che le osservazioni contengono 4 numeri. Essi sono:
- Posizione del carrello
- Velocità del carrello
- Angolo del palo
- Velocità di rotazione del palo

`rew` è la ricompensa che riceviamo a ogni passo. Puoi vedere che nell'ambiente CartPole si riceve 1 punto per ogni passo di simulazione, e l'obiettivo è massimizzare la ricompensa totale, ovvero il tempo in cui CartPole riesce a bilanciarsi senza cadere.

Durante l'apprendimento per rinforzo, il nostro obiettivo è allenare una **politica** $\pi$, che per ogni stato $s$ ci dirà quale azione $a$ intraprendere, quindi essenzialmente $a = \pi(s)$.

Se desideri una soluzione probabilistica, puoi pensare alla politica come a un insieme di probabilità per ogni azione, ovvero $\pi(a|s)$ rappresenterebbe la probabilità che dovremmo intraprendere l'azione $a$ nello stato $s$.

## Metodo del Gradiente di Politica

Nel più semplice algoritmo di RL, chiamato **Gradiente di Politica**, alleneremo una rete neurale per prevedere la prossima azione.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Alleneremo la rete eseguendo molti esperimenti e aggiornando la nostra rete dopo ogni esecuzione. Definiamo una funzione che eseguirà l'esperimento e restituirà i risultati (il cosiddetto **traccia**) - tutti gli stati, le azioni (e le loro probabilità consigliate) e le ricompense:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Puoi eseguire un episodio con una rete non addestrata e osservare che il premio totale (ovvero la durata dell'episodio) è molto basso:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Uno degli aspetti complicati dell'algoritmo di policy gradient è l'uso delle **ricompense scontate**. L'idea è che calcoliamo il vettore delle ricompense totali a ogni passo del gioco e, durante questo processo, scontiamo le ricompense iniziali utilizzando un coefficiente $gamma$. Normalizziamo anche il vettore risultante, perché lo useremo come peso per influenzare il nostro allenamento:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Ora iniziamo l'addestramento vero e proprio! Eseguiremo 300 episodi e, in ciascun episodio, faremo quanto segue:

1. Esegui l'esperimento e raccogli la traccia.
2. Calcola la differenza (`gradients`) tra le azioni intraprese e le probabilità previste. Minore è la differenza, più siamo sicuri di aver preso l'azione corretta.
3. Calcola le ricompense scontate e moltiplica i gradienti per le ricompense scontate - questo assicurerà che i passi con ricompense più alte abbiano un impatto maggiore sul risultato finale rispetto a quelli con ricompense più basse.
4. Le azioni target previste per la nostra rete neurale saranno in parte prese dalle probabilità previste durante l'esecuzione e in parte dai gradienti calcolati. Utilizzeremo il parametro `alpha` per determinare in che misura considerare gradienti e ricompense - questo è chiamato *tasso di apprendimento* dell'algoritmo di rinforzo.
5. Infine, addestriamo la nostra rete sugli stati e sulle azioni previste, e ripetiamo il processo.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Ora eseguiamo l'episodio con il rendering per vedere il risultato:


In [ ]:
_ = run_episode(render=True)

Speriamo che tu possa vedere che ora l'asta riesce a bilanciarsi piuttosto bene!

## Modello Actor-Critic

Il modello Actor-Critic è un ulteriore sviluppo dei gradienti di politica, in cui costruiamo una rete neurale per apprendere sia la politica che le ricompense stimate. La rete avrà due output (o puoi considerarla come due reti separate):
* **Actor** raccomanderà l'azione da intraprendere fornendoci la distribuzione di probabilità dello stato, come nel modello a gradiente di politica.
* **Critic** stimerebbe quale sarebbe la ricompensa derivante da quelle azioni. Restituisce le ricompense totali stimate nel futuro nello stato dato.

Definiamo un modello di questo tipo:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Avremmo bisogno di modificare leggermente le nostre funzioni `discounted_rewards` e `run_episode`:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Ora eseguiremo il ciclo principale di addestramento. Utilizzeremo il processo di addestramento manuale della rete calcolando le funzioni di perdita appropriate e aggiornando i parametri della rete:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Conclusioni

Abbiamo visto due algoritmi di apprendimento per rinforzo in questa demo: il semplice policy gradient e l'attore-critico più sofisticato. Puoi notare che questi algoritmi operano con nozioni astratte di stato, azione e ricompensa - per questo motivo possono essere applicati a ambienti molto diversi.

L'apprendimento per rinforzo ci permette di apprendere la strategia migliore per risolvere un problema semplicemente osservando la ricompensa finale. Il fatto che non abbiamo bisogno di dataset etichettati ci consente di ripetere le simulazioni molte volte per ottimizzare i nostri modelli. Tuttavia, ci sono ancora molte sfide nell'apprendimento per rinforzo, che potresti approfondire se decidi di concentrarti maggiormente su questa interessante area dell'IA.



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
